In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 🚀 Install Pycaret

In [ ]:
!pip install pycaret

# 🚀 Import Libraries

In [ ]:
from pycaret.classification import *

# 🚀 Load training and testing data

## Load training data

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
train.head()

## Load testing data

In [ ]:
test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
test.head()

## Load sample_subnmission file

In [ ]:
sample_submission = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv')
sample_submission.head()

# 🚀 Spliting training data into train and test

In [ ]:
data = train.sample(frac=0.9, random_state=786)
data_unseen = train.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

# 🚀 Create Model pipeline

In [ ]:
exp_mclf101 = setup(data = data, 
                    ignore_features = ['id'], 
                    numeric_features=['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
                                      'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
                                      'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14',
                                      'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19',
                                      'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24',
                                      'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_29',
                                      'feature_30', 'feature_31', 'feature_32', 'feature_33', 'feature_34',
                                      'feature_35', 'feature_36', 'feature_37', 'feature_38', 'feature_39',
                                      'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44',
                                      'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_49'],
                 #   pca= True,
                #    pca_method = 'incremental',
                    fix_imbalance = True,
                    data_split_stratify = True,
                    fold_shuffle= True,
                  #  use_gpu= True,
                    target = 'target',
                    silent = True,
                    session_id=123)

# 🚀 Compare different models

In [ ]:
# best = compare_models()
top3 = compare_models(exclude=['gbc'])

In [ ]:
lightgbm = create_model('lightgbm')

In [ ]:
rf = create_model('rf')

In [ ]:
catboost = create_model('catboost')

In [ ]:
et = create_model('et')

In [ ]:
ada = create_model('ada')

# 🚀 Blending different models

In [ ]:
#blender = blend_models(top3)
blender = blend_models(estimator_list = [lightgbm,rf,et,ada,catboost],choose_better=True, optimize='F1', method = 'soft')

# 🚀 Plot AUC curve

In [ ]:
plot_model(blender, plot = 'auc')

# 🚀 Plot Confusion matrix

In [ ]:
plot_model(blender, plot = 'confusion_matrix')

# 🚀 Plot class report

In [ ]:
plot_model(blender, plot = 'class_report')

# 🚀 Plot boundry chart

In [ ]:
plot_model(blender, plot='boundary')

# 🚀 Plot error chart

In [ ]:
plot_model(blender, plot = 'error')

# 🚀 Evaluate model

In [ ]:
evaluate_model(blender)

# 🚀 Predict on full data

In [ ]:
predict_model(blender)

# 🚀 Finalize the model

In [ ]:
final_blend = finalize_model(blender)

In [ ]:
print(final_blend)

# 🚀 Predict on unseen data

In [ ]:
unseen_predictions = predict_model(final_blend, data=data_unseen)
unseen_predictions.head()

# 🚀 Save model

In [ ]:
save_model(final_blend,'Final blend Model 07May2021')

# 🚀 Load Model

In [ ]:
saved_final_blend = load_model('Final blend Model 07May2021')

# 🚀 Predicting on the prediction dataset

In [ ]:
new_prediction = predict_model(saved_final_blend, raw_score=True, data=test)
new_prediction.head()

# 🚀 Submission

In [ ]:
new_prediction.to_csv('submission1.csv', index= False)